In [2]:
import numpy as np
import altair as alt
import pandas as pd

In [3]:
# Parameters for the Simulation 

past_sprints = [10, 8, 20, 11, 15]
goal = 50
num_sim_runs = 60

In [4]:
#Simulate how many sprints are required to hit the target number of story points.
def burn_up(target):
    burnup = []
    total = 0
    while total < target:
        sprint = np.random.choice(past_sprints)
        total += sprint
        burnup.append(sprint)
    return burnup
        


In [5]:

def simulate(num_sim_runs, story_point_goal):
    return [burn_up(story_point_goal) for _ in range(num_sim_runs)]


    

In [6]:

def cum_prob(histogram, total):
    return np.cumsum(histogram / total)

In [7]:
def inverse_cdf(cdf, percent):
    pos = 0
    for (i, value) in enumerate(cdf):
        if value >= percent:
            pos = i
            break
    return pos - (cdf[pos] - percent) / (cdf[pos] - cdf[pos-1])


In [8]:
data = simulate(num_sim_runs, goal)

In [9]:
counts = list(map(len, data))

In [10]:
(hist, _) = np.histogram(counts, bins=range(max(counts)+2))
hist

array([ 0,  0,  0, 10, 28, 20,  2])

In [11]:
probs = cum_prob(hist, num_sim_runs)

In [12]:
fiftieth = inverse_cdf(probs, .5)
fiftieth

3.7142857142857144

In [13]:
eightyfifth = inverse_cdf(probs, .85)
eightyfifth

4.65

In [14]:
sprint = []
runs = []
burnups = []

for (i, run) in enumerate(data):
    for (j, k) in enumerate(np.cumsum(run)):
        runs.append(f"run{i}")
        sprint.append(j+1)
        burnups.append(k)
        
df = pd.DataFrame({
    'Sprint': sprint,
    'Runs': runs,
    'Burnups': burnups
})
df
    

,Sprint,Runs,Burnups
0,1,run0,11
1,2,run0,22
2,3,run0,32
3,4,run0,47
4,5,run0,57
...,...,...,...
249,5,run58,58
250,1,run59,8
251,2,run59,23
252,3,run59,31


In [15]:

lines = alt.Chart(df).properties(width=800, height=500).mark_line().encode(
    x=alt.X('Sprint:O', axis=alt.Axis(labelAngle=0)),
    y='Burnups:Q',
    color= alt.Color('Runs', legend=None)
)

goal_line = alt.Chart(pd.DataFrame({'Sprint Goal': [50]})).mark_rule().encode(y='Sprint Goal')

fiftieth_line = alt.Chart(pd.DataFrame({'50th': [fiftieth]})).mark_rule().encode(x='50th')

lines + goal_line


alt.LayerChart(...)

In [19]:
import plotly.express as px

fig = px.line(df, x="Sprint", y="Burnups", color="Runs", line_group="Runs")
fig.show()